In [94]:
from entsoe import EntsoePandasClient
import pandas as pd
import matplotlib.pyplot as plt

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [95]:
start = pd.Timestamp('20230314', tz='Europe/Amsterdam')
end = pd.Timestamp('20230315', tz='Europe/Amsterdam')

countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

gen_types = [
    'Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 
    'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 
    'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other',
    'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore'
]

standardized_gen_types = [
    'biomass', 'brown_coal', 'coal_gas', 'natural_gas', 'hard_coal', 'oil', 'oil_shale', 'peat', 'geothermal', 
    'hydro_pumped_storage', 'hydro_run_of_river', 'hydro_reservoir', 'marine', 'nuclear', 'other', 'other_renewable',
    'solar', 'waste', 'wind_offshore', 'wind_onshore'
]

In [96]:
def standardize_generation_data(df_gen):
    for gen_type in gen_types:
        if gen_type not in df_gen:
            df_gen[gen_type] = 0.0

    for gen_type in gen_types:
        if (gen_type, 'Actual Consumption') in df_gen:
            df_gen = df_gen.drop(columns=[(gen_type, 'Actual Consumption')], axis=1)

    df_gen = df_gen[gen_types]
    df_gen.columns = standardized_gen_types

    df_gen = df_gen.tz_localize(None)

    df_gen = df_gen.resample("H").agg(["mean"])
    df_gen.columns = standardized_gen_types
    
    return df_gen

In [98]:
with pd.ExcelWriter("generation.xlsx") as writer:  
    for country_code in countries:
        df_gen = client.query_generation(country_code, start=start,end=end)
        print(country_code)

        df_gen = standardize_generation_data(df_gen)    
        df_gen.to_excel(writer, sheet_name=country_code) 

AT
BE
CZ
DE_LU
FR
HR
HU
NL
PL
RO
SI
SK
